In [1]:
from src.sim_utils.sim import Team, GameState, Player
import pandas as pd
import yaml
import matplotlib.pyplot as plt

ImportError: cannot import name 'WithDropout' from 'sim_utils.modeling' (D:\Github\NFL_sim\src\sim_utils\modeling.py)

In [ ]:
from utils import betting

In [ ]:
with open("models/feature_config.yaml", "r") as file:
    config = yaml.safe_load(file)

vik = Team("DET", 2024, 17)
det = Team("CHI", 2024, 17)
game = GameState(vik, det, config, **{"wind": 5, "temp": 90, "spread_line": -12.5})
game.play_game()
game.game_results(df=True).groupby(["team_name"]).sum()[
    ["receiving_yards", "rushing_yards"]
]


CHI has won 34 - 28


,receiving_yards,rushing_yards
team_name,,
CHI,409,276
DET,260,12


In [ ]:
df = (
    game.game_results(df=True)
    .groupby(["team_name"])
    .sum()[["receiving_yards", "rushing_yards", "carries"]]
)
df["ypc"] = df["rushing_yards"] / df["carries"]
df

,receiving_yards,rushing_yards,carries,ypc
team_name,,,,
CHI,409,276,92,3.0
DET,260,12,5,2.4


In [ ]:
def run_sim(team1, team2, num_sims, config=config):
    results = []
    scores = []
    vik = Team(team1, 2024, 13)
    det = Team(team2, 2024, 13)
    game = GameState(vik, det, config, **{"wind": 5, "temp": 60, "spread_line": 6.5})
    for i in range(num_sims):
        game.start_game()
        game.play_game()
        scores.append((game.home.score, game.away.score))
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results, scores


results, scores = run_sim(
    "DET", "CHI", 10
)  # 28.8 -> 22.8 by pulling game initialization out of loop

DET has won 20 - 13
CHI has won 35 - 21
DET has won 35 - 20
CHI has won 28 - 20
CHI has won 35 - 24
DET has won 14 - 9
CHI has won 24 - 20
DET has won 27 - 14
DET has won 28 - 21
DET has won 42 - 20


In [ ]:
df = pd.json_normalize(results)
df.groupby(["name", "id"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].agg(["median", "mean"]).sort_values(by=[("receiving_yards", "mean")], ascending=False)

receiving_yards       targets      receptions  \
                                      median  mean  median mean     median   
name              id                                                         
Sam LaPorta       00-0039065            74.5  88.4     8.5  8.9        5.5   
Amon-Ra St. Brown 00-0036963            86.0  87.0     8.0  7.9        4.5   
Jahmyr Gibbs      00-0039139            66.0  78.4     7.0  6.8        4.5   
Cole Kmet         00-0036290            38.5  60.6     5.0  5.3        2.5   
Rome Odunze       00-0039919            52.5  56.9     4.5  4.9        2.5   
D.J. Moore        00-0034827            40.5  50.5     6.5  6.7        3.0   
Keenan Allen      00-0030279            40.0  40.4     5.5  6.5        3.0   
Tim Patrick       00-0033375            15.5  32.3     3.0  3.1        1.5   
D'Andre Swift     00-0036275            30.0  30.3     4.0  4.2        2.5   
David Montgomery  00-0035685            19.5  29.4     4.5  4.2        3.0   
Allen Robinson    00-0031428             0.5  17.0     1.0  1.3        0.5   
Marcedes Lewis    00-0024243             1.5   9.9     0.5  0.8        0.5   
Brock Wright      00-0036754             0.0   9.4     1.5  1.4        1.0   
Roschon Johnson   00-0039021             0.0   8.7     1.0  1.2        0.0   
Gerald Everett    00-0033895             0.0   6.2     0.0  0.4        0.0   
DeAndre Carter    00-0031763             0.0   0.8     0.5  0.6        0.0   
Caleb Williams    00-0039918             0.0   0.0     0.0  0.0        0.0   
Jared Goff        00-0033106             0.0   0.0     0.0  0.0        0.0   
Craig Reynolds    00-0035567             0.0   0.0     0.0  0.0        0.0   
Sione Vaki        00-0039364             0.0   0.0     0.0  0.0        0.0   
Travis Homer      00-0035594             0.0   0.0     0.0  0.0        0.0   

                                  air_yards       completions       attempts  \
                             mean    median  mean      median  mean   median   
name              id                                                           
Sam LaPorta       00-0039065  5.5      38.0  49.0         0.0   0.0      0.0   
Amon-Ra St. Brown 00-0036963  4.4      30.0  44.1         0.0   0.0      0.0   
Jahmyr Gibbs      00-0039139  4.6       9.5  15.3         0.0   0.0      0.0   
Cole Kmet         00-0036290  3.1      18.5  35.0         0.0   0.0      0.0   
Rome Odunze       00-0039919  3.0      23.0  36.3         0.0   0.0      0.0   
D.J. Moore        00-0034827  3.3       8.0  11.8         0.0   0.0      0.0   
Keenan Allen      00-0030279  2.9       5.0  18.6         0.0   0.0      0.0   
Tim Patrick       00-0033375  1.5       4.0  17.1         0.0   0.0      0.0   
D'Andre Swift     00-0036275  2.4       7.5  11.0         0.0   0.0      0.0   
David Montgomery  00-0035685  2.7      -3.5  -0.8         0.0   0.0      0.0   
Allen Robinson    00-0031428  0.8       0.0   7.4         0.0   0.0      0.0   
Marcedes Lewis    00-0024243  0.6       0.0   2.0         0.0   0.0      0.0   
Brock Wright      00-0036754  0.7       0.0   1.8         0.0   0.0      0.0   
Roschon Johnson   00-0039021  0.5       0.0   0.4         0.0   0.0      0.0   
Gerald Everett    00-0033895  0.4       0.0   4.4         0.0   0.0      0.0   
DeAndre Carter    00-0031763  0.2       0.0   0.2         0.0   0.0      0.0   
Caleb Williams    00-0039918  0.0       0.0   0.0        16.0  16.4     30.5   
Jared Goff        00-0033106  0.0       0.0   0.0        18.0  20.2     32.5   
Craig Reynolds    00-0035567  0.0       0.0   0.0         0.0   0.0      0.0   
Sione Vaki        00-0039364  0.0       0.0   0.0         0.0   0.0      0.0   
Travis Homer      00-0035594  0.0       0.0   0.0         0.0   0.0      0.0   

                                   rushing_yards       carries        
                              mean        median  mean  median  mean  
name              id                                                  
Sam LaPorta       00-0

In [ ]:
df.groupby(["team_name", "game_number"])[
    [
        "receiving_yards",
        "targets",
        "receptions",
        "air_yards",
        "completions",
        "attempts",
        "rushing_yards",
        "carries",
    ]
].sum().groupby(["team_name"]).mean().sort_values(
    by=["receiving_yards"], ascending=False
)

,receiving_yards,targets,receptions,air_yards,completions,attempts,rushing_yards,carries
team_name,,,,,,,,
CAR,393.4,34.9,22.3,190.7,22.3,34.9,142.9,27.7
PHI,340.0,33.2,19.7,177.1,19.7,33.2,164.4,35.5


In [ ]:
sched = pd.read_csv("week_13.csv", index_col=0)
sched.iloc[11]

game_id           2024_13_TB_CAR
season                      2024
week                          13
home_team                    CAR
away_team                     TB
home_moneyline             205.0
spread_line                 -6.5
total_line                  46.5
temp                        50.0
wind                         8.0
Name: 6896, dtype: object

In [ ]:
def run_sim(num_sims, sched, config=config):
    results = []
    scores = []
    for i in range(num_sims):
        vik = Team(sched["home_team"], sched["season"], sched["week"])
        det = Team(sched["away_team"], sched["season"], sched["week"])
        game = GameState(vik, det, config, **sched)
        game.start_game()
        game.play_game()
        scores.append((game.home.score, game.away.score))
        res = vik.game_results() + det.game_results()
        for result in res:
            result["game_number"] = i
        results += res
    return results, scores


# results, scores = run_sim('KC','LV',100)

In [ ]:
results = []
test = True
if test:
    for t in [11]:
        # for t in range(len(sched)):
        game1 = sched.iloc[t].to_dict()
        try:
            results, scores = run_sim("KC", "LV", 100, game1)
        except Exception as _:
            print("oops")

CAR has won 13 - 7
CAR has won 31 - 7
CAR has won 30 - 6
CAR has won 26 - 17
TB has won 28 - 24
CAR has won 34 - 27
TB has won 24 - 14
TB has won 27 - 20
TB has won 28 - 24
CAR has won 24 - 20
CAR has won 17 - 13
CAR has won 21 - 13
CAR has won 40 - 14
CAR has won 24 - 14
TB has won 16 - 16
TB has won 20 - 19
TB has won 23 - 21
CAR has won 26 - 17
TB has won 21 - 17
TB has won 26 - 17
TB has won 17 - 17
CAR has won 13 - 10
CAR has won 24 - 9
CAR has won 45 - 17
CAR has won 44 - 17
TB has won 31 - 23
CAR has won 31 - 21
TB has won 20 - 7
TB has won 17 - 17
CAR has won 23 - 17
TB has won 27 - 6
CAR has won 28 - 27
TB has won 21 - 16
TB has won 24 - 6
TB has won 24 - 14
CAR has won 41 - 16
CAR has won 23 - 17
CAR has won 41 - 38
TB has won 31 - 14
TB has won 20 - 17
TB has won 34 - 14
CAR has won 20 - 10
TB has won 10 - 10
TB has won 31 - 10
CAR has won 34 - 17
TB has won 24 - 23
TB has won 21 - 7
TB has won 26 - 26
CAR has won 41 - 16
CAR has won 20 - 9
TB has won 24 - 24
CAR has won 24 

In [ ]:
weekly = pd.read_csv("data/weekly.csv", index_col=0)
pit = weekly.loc[
    (weekly.week == 12)
    & (weekly.season == 2024)
    & ((weekly.recent_team == "PIT") | (weekly.recent_team == "CLE"))
]